### Basic Imports

In [2]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import torchvision

import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

In [3]:
# random seed
SEED = 1 
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 30
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### cifar10在之前作业已得到数据，故不运行


### cifar100


In [5]:
transform_cifar100_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
#裁剪到相同scale

transform_cifar100_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_cifar100_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_cifar100_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm']




Files already downloaded and verified
Files already downloaded and verified


In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
model = ConvNet()
model=model.cuda()

In [8]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

In [9]:
criterion =nn.CrossEntropyLoss()

In [10]:
def train_batch(model, image, target):
    output =  model(image)
    loss = criterion(output, target)
    return output, loss

In [11]:

def test_batch(model, image, target):
    output =  model(image)
    loss = criterion(output, target)
    return output, loss

In [12]:
CRtraining_loss = []
CRtraining_acc = []
CRtesting_loss = []
CRtesting_acc = []
CRtrain_preds = []
CRtrain_targets = []
CRtest_preds = []
CRtest_targets = []
CRtrain_f1_scores = []
CRtest_f1_scores = []
CRlayer_gradients = [[] for _ in range(len(list(model.parameters())))]

for epoch in range(NUM_EPOCHS):
    model.train()
    torch.cuda.empty_cache()

    ##########################
    ### Training
    ##########################

    running_cls_loss = 0.0
    running_cls_corrects = 0

    for batch_idx, (image, target) in enumerate(train_dataloader):
        image = image.to(device)
        target = target.to(device)

        # train model
        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)

        CRtrain_preds.extend(preds.cpu().tolist())
        CRtrain_targets.extend(target.cpu().tolist())

        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while training')
        running_cls_loss += loss.item()
        running_cls_corrects += torch.sum(preds == target.data)

        loss.backward()
        for i, (name, param) in enumerate(model.named_parameters()):
            if param.grad is not None:
                CRlayer_gradients[i].append(param.grad.abs().mean().item())

        optimizer.step()
        optimizer.zero_grad()

    CRtrain_recall = recall_score(CRtrain_targets, CRtrain_preds, average='weighted')
    CRtrain_f1 = f1_score(CRtrain_targets, CRtrain_preds, average='weighted')
    CRtrain_f1_scores.append(CRtrain_f1)
    CRepoch_loss = running_cls_loss / len(train_set)
    CRepoch_acc = running_cls_corrects.double() / len(train_set)

    print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {CRepoch_loss:.4f} Acc: {CRepoch_acc:.4f}')
    print(f'Train Recall: {CRtrain_recall:.4f} F1 Score: {CRtrain_f1:.4f}')
    CRtraining_loss.append(CRepoch_loss)
    CRtraining_acc.append(CRepoch_acc.cpu().detach().numpy())
    scheduler.step()
    if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch + 1) == NUM_EPOCHS:
        print('Begin test......')
        model.eval()

        CRval_loss = 0.0
        CRval_corrects = 0
        CRtest_preds = []
        CRtest_targets = []

        for batch_idx, (image, target) in enumerate(test_dataloader):
            image = image.to(device)
            target = target.to(device)

            # test model
            outputs, loss = test_batch(model, image, target)
            _, preds = torch.max(outputs, 1)
            CRtest_preds.extend(preds.cpu().tolist())
            CRtest_targets.extend(target.cpu().tolist())

            CRval_loss += loss.item()
            CRval_corrects += torch.sum(preds == target.data)

        CRtest_recall = recall_score(CRtest_targets, CRtest_preds, average='weighted')
        CRtest_f1 = f1_score(CRtest_targets, CRtest_preds, average='weighted')
        CRtest_f1_scores.append(CRtest_f1)
        CRval_loss = CRval_loss / len(test_set)
        CRval_acc = CRval_corrects.double() / len(test_set)
        print(f'Test Loss: {CRval_loss:.4f} Acc: {CRval_acc:.4f}')
        print(f'Test Recall: {CRtest_recall:.4f} F1 Score: {CRtest_f1:.4f}')
        CRtesting_loss.append(CRval_loss)
        CRtesting_acc.append(CRval_acc.cpu().detach().numpy())

Epoch: 1/30 Train Loss: 0.0156 Acc: 0.2512
Train Recall: 0.2512 F1 Score: 0.2377
Begin test......
Test Loss: 0.0142 Acc: 0.3326
Test Recall: 0.3326 F1 Score: 0.3041
Epoch: 2/30 Train Loss: 0.0144 Acc: 0.3095
Train Recall: 0.2804 F1 Score: 0.2704
Begin test......
Test Loss: 0.0139 Acc: 0.3410
Test Recall: 0.3410 F1 Score: 0.3338
Epoch: 3/30 Train Loss: 0.0142 Acc: 0.3248
Train Recall: 0.2952 F1 Score: 0.2862
Begin test......
Test Loss: 0.0138 Acc: 0.3389
Test Recall: 0.3389 F1 Score: 0.3161
Epoch: 4/30 Train Loss: 0.0140 Acc: 0.3274
Train Recall: 0.3032 F1 Score: 0.2950
Begin test......
Test Loss: 0.0132 Acc: 0.3690
Test Recall: 0.3690 F1 Score: 0.3342
Epoch: 5/30 Train Loss: 0.0140 Acc: 0.3300
Train Recall: 0.3086 F1 Score: 0.3006
Begin test......
Test Loss: 0.0132 Acc: 0.3695
Test Recall: 0.3695 F1 Score: 0.3572
Epoch: 6/30 Train Loss: 0.0132 Acc: 0.3669
Train Recall: 0.3183 F1 Score: 0.3105
Begin test......
Test Loss: 0.0126 Acc: 0.4093
Test Recall: 0.4093 F1 Score: 0.3908
Epoch: 7/3

### MNIST

In [14]:

transform_mnist_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

transform_mnist_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载 MNIST 训练数据集
train_set = torchvision.datasets.MNIST(root='../data', train=True,
                                       download=True, transform=transform_mnist_train)
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=2)

# 加载 MNIST 测试数据集
test_set = torchvision.datasets.MNIST(root='../data', train=False,
                                      download=True, transform=transform_mnist_test)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=2)

# 类别名称
class_names = [str(i) for i in range(10)]


100%|██████████| 9912422/9912422 [00:00<00:00, 94523550.90it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 33658153.33it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21108653.27it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5436794.74it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [16]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3)  # 修改为 1 个输入通道
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)
        # 调整全连接层的输入尺寸
        self.fc1 = nn.Linear(8 * 5 * 5, 32)  # 注意这里的尺寸可能需要调整
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 5 * 5)  # 注意这里的尺寸可能需要调整
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [17]:
model = ConvNet()
model=model.cuda()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

In [18]:
criterion =nn.CrossEntropyLoss()

In [19]:
CRtraining_loss = []
CRtraining_acc = []
CRtesting_loss = []
CRtesting_acc = []
CRtrain_preds = []
CRtrain_targets = []
CRtest_preds = []
CRtest_targets = []
CRtrain_f1_scores = []
CRtest_f1_scores = []
CRlayer_gradients = [[] for _ in range(len(list(model.parameters())))]

for epoch in range(NUM_EPOCHS):
    model.train()
    torch.cuda.empty_cache()

    ##########################
    ### Training
    ##########################

    running_cls_loss = 0.0
    running_cls_corrects = 0

    for batch_idx, (image, target) in enumerate(train_dataloader):
        image = image.to(device)
        target = target.to(device)

        # train model
        outputs, loss = train_batch(model, image, target)
        _, preds = torch.max(outputs, 1)

        CRtrain_preds.extend(preds.cpu().tolist())
        CRtrain_targets.extend(target.cpu().tolist())

        loss_data = loss.data.item()
        if np.isnan(loss_data):
            raise ValueError('loss is nan while training')
        running_cls_loss += loss.item()
        running_cls_corrects += torch.sum(preds == target.data)

        loss.backward()
        for i, (name, param) in enumerate(model.named_parameters()):
            if param.grad is not None:
                CRlayer_gradients[i].append(param.grad.abs().mean().item())

        optimizer.step()
        optimizer.zero_grad()

    CRtrain_recall = recall_score(CRtrain_targets, CRtrain_preds, average='weighted')
    CRtrain_f1 = f1_score(CRtrain_targets, CRtrain_preds, average='weighted')
    CRtrain_f1_scores.append(CRtrain_f1)
    CRepoch_loss = running_cls_loss / len(train_set)
    CRepoch_acc = running_cls_corrects.double() / len(train_set)

    print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {CRepoch_loss:.4f} Acc: {CRepoch_acc:.4f}')
    print(f'Train Recall: {CRtrain_recall:.4f} F1 Score: {CRtrain_f1:.4f}')
    CRtraining_loss.append(CRepoch_loss)
    CRtraining_acc.append(CRepoch_acc.cpu().detach().numpy())
    scheduler.step()
    if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch + 1) == NUM_EPOCHS:
        print('Begin test......')
        model.eval()

        CRval_loss = 0.0
        CRval_corrects = 0
        CRtest_preds = []
        CRtest_targets = []

        for batch_idx, (image, target) in enumerate(test_dataloader):
            image = image.to(device)
            target = target.to(device)

            # test model
            outputs, loss = test_batch(model, image, target)
            _, preds = torch.max(outputs, 1)
            CRtest_preds.extend(preds.cpu().tolist())
            CRtest_targets.extend(target.cpu().tolist())

            CRval_loss += loss.item()
            CRval_corrects += torch.sum(preds == target.data)

        CRtest_recall = recall_score(CRtest_targets, CRtest_preds, average='weighted')
        CRtest_f1 = f1_score(CRtest_targets, CRtest_preds, average='weighted')
        CRtest_f1_scores.append(CRtest_f1)
        CRval_loss = CRval_loss / len(test_set)
        CRval_acc = CRval_corrects.double() / len(test_set)
        print(f'Test Loss: {CRval_loss:.4f} Acc: {CRval_acc:.4f}')
        print(f'Test Recall: {CRtest_recall:.4f} F1 Score: {CRtest_f1:.4f}')
        CRtesting_loss.append(CRval_loss)
        CRtesting_acc.append(CRval_acc.cpu().detach().numpy())

Epoch: 1/30 Train Loss: 0.0037 Acc: 0.8555
Train Recall: 0.8555 F1 Score: 0.8551
Begin test......
Test Loss: 0.0016 Acc: 0.9409
Test Recall: 0.9409 F1 Score: 0.9409
Epoch: 2/30 Train Loss: 0.0017 Acc: 0.9333
Train Recall: 0.8944 F1 Score: 0.8942
Begin test......
Test Loss: 0.0014 Acc: 0.9517
Test Recall: 0.9517 F1 Score: 0.9519
Epoch: 3/30 Train Loss: 0.0015 Acc: 0.9407
Train Recall: 0.9098 F1 Score: 0.9097
Begin test......
Test Loss: 0.0019 Acc: 0.9323
Test Recall: 0.9323 F1 Score: 0.9325
Epoch: 4/30 Train Loss: 0.0015 Acc: 0.9433
Train Recall: 0.9182 F1 Score: 0.9181
Begin test......
Test Loss: 0.0012 Acc: 0.9559
Test Recall: 0.9559 F1 Score: 0.9559
Epoch: 5/30 Train Loss: 0.0014 Acc: 0.9448
Train Recall: 0.9235 F1 Score: 0.9234
Begin test......
Test Loss: 0.0014 Acc: 0.9481
Test Recall: 0.9481 F1 Score: 0.9482
Epoch: 6/30 Train Loss: 0.0010 Acc: 0.9596
Train Recall: 0.9295 F1 Score: 0.9295
Begin test......
Test Loss: 0.0010 Acc: 0.9640
Test Recall: 0.9640 F1 Score: 0.9640
Epoch: 7/3